In [1]:
import pandas as pd
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

In [2]:
# conectarse a bbdd mysql
with open ('password.txt') as f:
   pwd=str(f.readline()).strip()

conexion = pymysql.connect(host='localhost',
                           database = 'henry_pi',
                           user= 'root',
                           password=pwd)

cursor = conexion.cursor()

In [3]:
# se le pasa el nombre de una tabla contenida en una base de datos a la cual se esta conectado
# devuelve un dataframe
def extraer_datos(tabla:str):
    cursor.execute("SELECT * FROM {}".format(tabla))
    rows = cursor.fetchall()
    headers = [column[0] for column in cursor.description]
    df= pd.DataFrame(rows,columns=headers)
    return df


In [4]:
# extraer datos de bbdd
historico_velocidad_nacional=extraer_datos('historico_velocidad_nacional')
ingresos_trimestre=extraer_datos('ingresos_trimestre')
internet_accesos_velocidad_provincias=extraer_datos('internet_accesos_velocidad_provincias')
internet_penetracion_hogares=extraer_datos('internet_penetracion_hogares')
internet_tipo_nacional=extraer_datos('internet_tipo_nacional')
internet_tipo_provincias=extraer_datos('internet_tipo_provincias')
localidades_servicios=extraer_datos('localidades_servicios')

In [5]:
# unir los dataframes segun la estructura de los datos
nacionales = [historico_velocidad_nacional,ingresos_trimestre,internet_tipo_nacional]
provinciales = [internet_accesos_velocidad_provincias,internet_penetracion_hogares,internet_tipo_provincias]
locales=[localidades_servicios]

In [6]:
# deshacerse de algunas columnas innecesarias
historico_velocidad_nacional.drop(columns='trimestre_d',inplace=True)
ingresos_trimestre.drop(columns='periodo',inplace=True)

In [7]:
df_nacionales = reduce(lambda left,right:pd.merge(left,right, on=['año','trimestre'],how='outer'),nacionales) # uso reduce para hacer un join de los dataframes

In [8]:
internet_tipo_provincias['trimestre']=internet_tipo_provincias.trimestre.astype(int)
internet_accesos_velocidad_provincias.rename(columns={'otros':'otras_velocidades'},inplace=True)
#internet_tipo_provincias.drop(columns='total',inplace=True)

In [9]:
df_provinciales = reduce(lambda left,right:pd.merge(left,right,on=['año','trimestre','provincia'],how='outer'),provinciales)

In [10]:
df_locales = locales[0]

In [11]:
# latitud y longitud a  float
df_locales['latitud']=df_locales['latitud'].str.replace('..','.')
df_locales['latitud']=df_locales['latitud'].astype(float)


In [12]:
df_locales['longitud']=df_locales['longitud'].str.replace('..','.')
df_locales['longitud']=df_locales['longitud'].astype(float)
df_locales.rename(columns={'latitud':'latitude','longitud':'longitude'})

,provincia,partido,localidad,poblacion,adsl,cable_modem,dial_up,fibra_opt,satelital,wireless,telefonia_fija,3g,4g,link,latitude,longitude
0,BUENOS AIRES,Bahía Blanca,Villa Bordeau,0,NO,NO,NO,NO,NO,NO,NO,SI,SI,6056020,-38.647261,-62.320022
1,BUENOS AIRES,Bahía Blanca,Villa Espora,0,NO,NO,NO,NO,NO,NO,NO,NO,NO,6056020,-38.776507,-62.185183
2,BUENOS AIRES,Balcarce,Balcarce,38376,SI,SI,SI,SI,NO,SI,SI,SI,SI,6063010,-37.848278,-58.255167
3,BUENOS AIRES,Balcarce,Los Pinos,337,NO,NO,NO,NO,NO,SI,SI,NO,NO,6063020,-37.941206,-58.322592
4,BUENOS AIRES,Balcarce,Napaleofú,374,NO,NO,NO,SI,NO,SI,SI,SI,SI,6063030,-37.625498,-58.746186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4307,CHACO,Quitilipi,Quitilipi,24517,SI,NO,NO,SI,NO,SI,SI,SI,SI,22133010,-26.873207,-60.218524
4308,CHACO,Quitilipi,Villa El Palmar,879,NO,NO,NO,NO,NO,SI,NO,NO,SI,22133020,-26.455119,-60.164617
4309,CHACO,San Fernando,Barranqueras,54698,SI,SI,NO,SI,NO,SI,SI,SI,SI,22140010,-27.487774,-58.932742
4310,CHACO,San Fernando,Basail,1929,SI,SI,NO,NO,SI,SI,SI,SI,SI,22140020,-27.886866,-59.279100


In [13]:
df_locales.loc[df_locales['longitud']<-100,'longitud']=df_locales.longitud/1000
df_locales.loc[df_locales['latitud']<-100,'latitud']=df_locales.latitud/1000

In [14]:
df_nacionales.to_csv('datos_ordenados/nacionales.csv',index=False)
df_provinciales.to_csv('datos_ordenados/provinciales.csv',index=False)
df_locales.to_csv('datos_ordenados/locales.csv',index=False)